In [5]:
import sys
sys.path.append("../")
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import gpytorch
import timm
from adabelief_pytorch import AdaBelief

In [6]:
sys.path.append("../")
from estimator import ssge

In [7]:
estimator = ssge.SSGE(
    gpytorch.kernels.ScaleKernel(
        gpytorch.kernels.MaternKernel(
            ard_num_dims = 10
        )
    )
)

In [ ]:
dist = torch.distributions.multivariate_normal.MultivariateNormal(
    torch.zeros(10),
    torch.eye(10),
)

In [131]:
data = load_boston()

/home/nonconvexopt/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California h

In [102]:
def standarize(x):
    return (x - x.mean(0, keepdims = True)) / x.std(0, keepdims = True)

X = data['data']
Y = data['target']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
X_train = standarize(torch.from_numpy(X_train)).to(torch.float32)
Y_train = standarize(torch.from_numpy(Y_train)).to(torch.float32)
X_test = standarize(torch.from_numpy(X_test)).to(torch.float32)
Y_test = standarize(torch.from_numpy(Y_test)).to(torch.float32)

In [113]:
class GP_prior(gpytorch.models.ExactGP):
    def __init__(
        self,
        train_x: torch.Tensor,
        train_y: torch.Tensor,
        likelihood: gpytorch.likelihoods.Likelihood,
    ):
        super(GP_prior, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.cov_module = gpytorch.kernels.AdditiveKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(
                    ard_num_dims = train_x.shape[-1]
                )
            ),
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.PeriodicKernel(

                )
            )
        )

    def forward(self, x):
        return gpytorch.distributions.MultivariateNormal(
            self.mean_module(x),
            self.cov_module(x),
        )

In [124]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
likelihood.train()
model = GP_prior(
    X_train,
    Y_train,
    likelihood
)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [125]:
optimizer = torch.optim.LBFGS(model.parameters(), lr = 0.1)
iterator = tqdm.notebook.tqdm(range(50))
for ind in iterator:
    def closure():
        optimizer.zero_grad()
        loss = -mll(model(X_train), Y_train)
        loss.backward()
        return loss
    loss = optimizer.step(closure)
    iterator.set_postfix(loss=loss.item())

  0%|          | 0/50 [00:00<?, ?it/s]

<h3>Random inducing</h3>

<h3>Adversarial inducing</h3>

<h3>Test input as inducing</h3>

In [14]:
X_train.shape

torch.Size([404, 13])

In [15]:
Y_train.shape

torch.Size([404])

In [52]:
class ffn(nn.Module):
    def __init__(self):
        super(ffn, self).__init__()
        self.l1 = nn.Linear(13, 50, bias = True)
        self.l2 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

#net_fn = timm.create_model('mixer_b16_224', pretrained=True)
net_fn = ffn()

In [64]:
for name, module in net_fn.named_modules():
    if isinstance(module, nn.Linear):
        print(name, module)

l1 Linear(in_features=13, out_features=50, bias=True)
l2 Linear(in_features=50, out_features=1, bias=True)
